In [5]:
import pandas as pd
import numpy as np
import pyodbc
import warnings
import datetime
import logging
from node_rules import cgcr_oxygen_tree, cgcr_aluminium_prediction, vag_aluminium_prediction

In [6]:
warnings.filterwarnings('ignore')

# for logging purposes
code_start_time = datetime.datetime.now()
print("Start Time : ", code_start_time)

Start Time :  2021-10-21 11:18:11.226360


In [7]:
conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=STEELDNA;'
                     'Database=DeS;'
                     'UID=sa;'
                     'PWD=admin@123;'
                     'Trusted_Connection=no;')

cursor = conn.cursor()

In [8]:
def call_prediction_function(row):
    
    """
    This function calls either the cgcr/vag prediction function based on the grade type for each heat
    """
    
    if row['GRADE_TYPE'] == 'CG/CR':
        return cgcr_aluminium_prediction(row)
    
    elif row['GRADE_TYPE'] == 'VAG':
        return vag_aluminium_prediction(row)
    
    else:
        print("Unknown Grade")
        return None

In [ ]:
def al_prediction_main(run_heats):
    """
    Main function that is called whenever predictions are to be made
    """
    
    ######################### data imports ##############################
    
    # importing heat_analysis
    heat_analysis = pd.read_sql("select TOP(1000) * from heat_analysis order by MSG_TIME_STAMP desc", conn)
    heat_analysis = heat_analysis[heat_analysis['HEAT_NUMBER'].isin(run_heats)]
    
    # importing lf_heat_data
    lf_heat_data = pd.read_sql("select TOP(1000) * from lf_heat_data order by MSG_TIME_STAMP desc", conn)
    lf_heat_data = lf_heat_data[lf_heat_data['HEAT_NUMBER'].isin(run_heats)]
    
    # importing the grade mapping fact table
    grade_mapping = pd.read_sql("select * from grade_mapping", conn)
    
    
    ######################### data processing ##############################
    
    # calculating required columns
    lf_heat_data['final_O2_ppm'] = lf_heat_data['TAP_O2'] + lf_heat_data['O2AFTERCELOX']

    # converting ALBAR to tons
    lf_heat_data['ALBAR'] = lf_heat_data['ALBAR']/19

    # renaming the required columns
    lf_heat_data.rename(columns = {
        'GRADE_TYPE':'GRADE',
        'LM_START_WT':'LM_Start_Wt_lf',
        'FIRSTMEASTEMP':'1st_Probe_temp_lf',
        'ALBAR':'Al_Bar_lf',
        'LIME':'Lime_tap',
        'SIMN':'SiMn_tap',
        'LTA':'LTA_lf'
    }, inplace = True)
    
    # keeping only the required rows
    lf_heat_data = lf_heat_data.groupby(['HEAT_NUMBER','STATION']).agg({'MSG_TIME_STAMP':'max','GRADE': 'first','1st_Probe_temp_lf':'max', 'Al_Bar_lf':'max','LM_Start_Wt_lf':'max','LTA_lf':'max','Lime_tap':'max','SiMn_tap':'max','final_O2_ppm':'max'})
    lf_heat_data.reset_index(inplace = True)
    lf_heat_data.sort_values(by = ['MSG_TIME_STAMP'], ascending = False, inplace = True)
    lf_heat_data.drop_duplicates(subset = ['HEAT_NUMBER'], keep = 'first', inplace = True)
    
    # preprocessing heat_analysis
    # if there are multiple records for a heat, keeping only the latest one
    heat_analysis.drop_duplicates(subset = ['HEAT_NUMBER'], keep = 'first', inplace = True)

    # keeping only required columns
    heat_analysis = heat_analysis[['HEAT_NUMBER','AGGREGATE','AL_TOTAL','S','C','SI']]

    # renaming columns as per requirement
    heat_analysis.rename(columns = {'AL_TOTAL':'AL_chem_first', 'S':'S_chem_first','SI':'SI_chem_first','C':'C_chem_first'}, inplace = True)
    
    
    # merging sample chemistry data with lf data
    ads = heat_analysis.merge(lf_heat_data, on = ['HEAT_NUMBER'], how = 'inner')
    
    
    # ads preprocessing
    
    # constraints to avoid cases with data issues
    # constraint no.1
    # al_bar range is 15-25
    ads = ads[(ads['Al_Bar_lf'] >= 15) & (ads['Al_Bar_lf'] <= 25)]
    # constraint no.2
    # lm weight is between 170 - 205
    ads = ads[(ads['LM_Start_Wt_lf'] >= 170) & (ads['LM_Start_Wt_lf'] <= 205)]
    
    # filling null values with 0 for now, update to default values later
    ads.fillna(0, inplace = True)

    # adding loi perc as a column
    # this is a fixed value
    ads['LOI_perc'] = 6
    
    # mapping the grades to their grade type (CG/CR or VAG)
    # the grades that don't get mapped are billet heats, and so will not be run in this model
    ads = ads.merge(grade_mapping, on = ['GRADE'], how = 'inner')
    
    ################ AL Prediction #################################
    
    # predicting aluminium material
    ads['AL_KGS'] = ads.apply(call_prediction_function, axis = 1)

    # predicting aluminium material in meters
    ads['AL_MTS'] = ads['AL_KGS'] * (10/3)
    
    # keeping only required columns
    ads = ads[['HEAT_NUMBER','AGGREGATE','AL_KGS','AL_MTS']]
    
    return ads

In [4]:
ads

NameError: name 'ads' is not defined

164